In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pandas as pd

# import the data and the sam data for cleaning, change directory if needed
# the 1st must be the code violations, 2nd the sam addresses, and 3rd the building violstions
df1 = pd.read_csv("./code_enforcement_violations.csv")
df2 = pd.read_csv("./Live_Street_Address_Management_(SAM)_Addresses.csv")
df3 = pd.read_csv("./build_prop_violations.csv")

C:\Users\maste.DESKTOP-6HD7KEF\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,9,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
#Process takes in 3 databases and processes them
def Process(code_vio, prop_vio, sam_addr):
    # rename the columns in the sam dataset to prepare for merging
    sam_addr.rename(columns={'SAM_ADDRESS_ID': 'sam_id', 'FULL_ADDRESS': 'full_address'}, inplace=True)
    # select only the wanted columns
    sam_addr = sam_addr[['full_address', 'sam_id']]
    # fix sam_id in df3 to get rid of the decimal, just in case
    # we drop missing sam_id for all data
    prop_vio.dropna(subset = ['sam_id'], inplace = True)
    code_vio.dropna(subset = ['sam_id'], inplace = True)
    prop_vio['sam_id'] = prop_vio['sam_id'].astype(int)
    # merge the datasets
    merged_code_vio = pd.merge(code_vio, sam_addr, on=['sam_id'], how='left')
    merged_prop_vio = pd.merge(prop_vio, sam_addr, on=['sam_id'], how='left')
    # drop the empty addresses, caused by missing sam addresses from what I can tell
    merged_code_vio.dropna(subset = ['full_address'], inplace = True)
    merged_prop_vio.dropna(subset = ['full_address'], inplace = True)
    # final clean by dropping apartment numbers ex. 13 Commonwealth #1 -> 13 Commonwealth
    merged_code_vio['full_address'] = merged_code_vio['full_address'].str.split(' #')
    merged_prop_vio['full_address'] = merged_prop_vio['full_address'].str.split(' #')
    # more misc cleaning and final check that all addresses are good if needed
    merged_code_vio.dropna(subset = ['ticket_no'], inplace = True)
    # print(merged_prop_vio.isnull().sum()) uncomment if needed for checking
    # print(merged_code_vio.isnull().sum())
    return merged_code_vio, merged_prop_vio

In [7]:
# Example call/ Test run
processed_code_vio, processed_code_vio = Process(df1, df3, df2)
processed_code_vio

,case_no,ap_case_defn_key,status_dttm,status,code,value,description,violation_stno,violation_sthigh,violation_street,...,contact_city,contact_state,contact_zip,sam_id,latitude,longitude,location,geom_point_2249,geom_point_4326,full_address
0,V81124,1013,NaN,Open,300.15,NaN,"Boxes, Where Required",83,,Tyler,...,Boston,MA,02111,138849,42.349028,-71.061613,"(42.34902752521983, -71.06161347530092)",0101000020C9080000019211311EA4274198D3509EA386...,0101000020E6100000435AA579F1C351C08B1C17EFAC2C...,[83 Tyler St]
1,V91983,1013,NaN,Closed,121.2,NaN,Unsafe and Dangerous,302,NaN,Sumner,...,East Boston,MA,02128,132380,42.367678,-71.036580,"(42.367678491544524, -71.03658037747812)",0101000020C908000000B62102B2D82741433D7DE4FB93...,0101000020E6100000683C395557C251C02C50BC16102F...,[302 Sumner St]
2,V738256,1013,2024-02-05 15:22:17,Open,105.1,NaN,Failure to Obtain Permit,123,NaN,Nottinghill,...,BRIGHTON,MA,02135,104234,42.343920,-71.148581,"(42.343919640314574, -71.14858068184424)",0101000020C908000002D88A2084EC2641FCF2F677CA82...,0101000020E6100000A8558C5882C951C035CD0B8F052C...,[123 Nottinghill Rd]
3,V738250,1013,2024-02-05 15:14:55,Open,105.1,NaN,Failure to Obtain Permit,272,276,Hyde Park,...,JAMAICA PLAIN,MA,02130,77289,42.292150,-71.117581,"(42.292149607069526, -71.11758072813018)",0101000020C9080000FE960D5CA62E27410976B68C035E...,0101000020E6100000607D517186C751C091F387286525...,[272-276 Hyde Park Ave]
4,V679074,1013,2024-02-05 15:02:11,Open,116,NaN,Unsafe Structures,44,NaN,Longfellow,...,DORCHESTER,MA,02122,87143,42.303210,-71.064711,"(42.303209532712025, -71.06471058438673)",0101000020C90800007B0FAE443E9E2741F853040C0566...,0101000020E610000059E9DC3724C451C0B36AE991CF26...,[44 Longfellow St]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15515,V53463,1013,2009-12-04 10:13:49,Closed,110.1,NaN,Inspections,20,NaN,Becket,...,DORCHESTER,MA,02124,13054,42.281010,-71.064261,"(42.281009534087694, -71.06426063283733)",0101000020C9080000FD510E63849F27413919C95B3856...,0101000020E6100000391DA1D81CC451C0AB62D31EF823...,[20 Becket St]
15517,V53371,1013,2009-12-03 11:26:33,Closed,103,NaN,Maintenance,55,NaN,Fenwood,...,BOSTON,MA,02115,57589,42.335320,-71.107611,"(42.33531958775082, -71.10761060896684)",0101000020C9080000FB7AF89728432741A0C7249DC37C...,0101000020E6100000C68D9B17E3C651C08A8C93C0EB2A...,[55 Fenwood Rd]
15518,V53076,1013,2009-12-03 10:01:21,Open,110.1,NaN,Inspections,225,NaN,Chelsea,...,EAST BOSTON,MA,02128,30417,42.375199,-71.032310,"(42.375199485231036, -71.03231035124385)",0101000020C90800008233DFC597E12741E07D3C715999...,0101000020E6100000897A6F5F11C251C08C4567890630...,[225 Chelsea St]
15519,V53078,1013,2009-12-01 13:39:34,Open,110.1,NaN,Inspections,1670,NaN,Washington,...,BOSTON,MA,02118,144315,42.337380,-71.075271,"(42.33737954420211, -71.07527053204305)",0101000020C908000082103D0771872741842BE2E74F7E...,0101000020E61000008E5E7E3BD1C451C0FE57BE402F2B...,[1670 Washington St]
